In [1]:
import pandas as pd
import random
import numpy as np
import os
from copy import deepcopy
from utils.Visualization import plot_action_prob, plotConfig
import matplotlib.pyplot as plt

In [2]:
# foldername = "./Results/SimpleSatellite-setgoals-v4/"
foldername_init = "./Results/arbiter_test/EqualEta/"
foldername = foldername_init + "Variable_Voice/" # "Variable_Voice/" # "Overlapping/"
fig_size = (20,10)
agent_list = os.listdir(foldername)

agent_list_to_sort = []
for i, fol in enumerate(agent_list):
    val_list = fol.split("_")
    agent_list_to_sort.append(float(val_list[1])+float(val_list[3])) 
    
agent_list = [x for _, x in sorted(zip(agent_list_to_sort, agent_list))]
for i, fol in enumerate(agent_list):
    print(f"{i}: {fol}")

0: Voices_4_Overlap_0.0


In [3]:
foldername_specific = foldername + "/" + agent_list[0] + "/"
filename = foldername_specific + "/results.feather"
print(filename)

./Results/arbiter_test/EqualEta/Variable_Voice//Voices_4_Overlap_0.0//results.feather


In [4]:
results_df = pd.read_feather(filename)
results_df.head()

In [17]:
# v4 epi 4 Test 3
specify_epi = 4 #len(results_df)-1 # len(results_df)-1  # results_df["Reward"].idxmin() # results_df["Reward"].idxmax() # random.randint(0, len(results_df)) 
Telemetry_dict = {}
Config_dict = {}
length_epi = len(results_df["Pos"][specify_epi])
total_actions = len(results_df["action_list"][specify_epi])
total_targets = total_actions // 3
print(f"Total actions: {total_actions}")
print(f"Total targets: {total_targets}")

action_list = results_df["action_list"][specify_epi]
for k, v in results_df.items():
    # try:
    if k == "Reward" or k == "Memory Size" or k == "N_Repeat":
        Config_dict[k] = v[specify_epi]
    else:
        try:
            length_vector = len(v[specify_epi]) 
            if length_vector > len(action_list)+1:
                if length_vector == length_epi:
                    Telemetry_dict[k] = v[specify_epi]
                else:
                    extension = [np.zeros(len(v[specify_epi][length_vector-1]))]*(length_epi - length_vector)
                    Telemetry_dict[k] = list(v[specify_epi]) + extension
                
            else:
                Config_dict[k] = v[specify_epi]
        except:
            print(f"Error with {k}")
    #     print(f"Error with {k}")
    #     Config_dict[k] = v[specify_epi]

# Error with Reward
# Error with Memory Size
# Error with N_Repeat

print(f"Episode: {specify_epi}")
print(f"Reward: {results_df['Reward'][specify_epi]}")

Total actions: 61
Total targets: 20
Error with Voice_0
Error with Voice_0 Targets
Episode: 4
Reward: 27


In [8]:
DbConfig_dict = {}
for k, v in Config_dict.items():
    if k != "Reward":
        DbConfig_dict[k] = [v]
DbTelemetry_dict = {}
for k, v in Telemetry_dict.items():
    DbTelemetry_dict[k] = v
action_list = Config_dict["action_list"]   

print(f"Config: {len(action_list)}")

Config: 61


Config_db = pd.DataFrame(DbConfig_dict)
Config_db.to_csv(foldername + "Config.csv")
Telemetry_db = pd.DataFrame(DbTelemetry_dict)
Telemetry_db.to_csv(foldername + "Telemetry.csv")


# Plots

## Set up

In [ ]:
init_orbit = 0
mks = 8
n_orbits = 10

In [10]:
import matplotlib.pyplot as plt
total_steps = len(Telemetry_dict["Pos"])-1
tot_orbit = total_steps // 300
t_orbits = Telemetry_dict["Orbit"][-1]
dP = int(n_orbits*300)
memory_size = Config_dict["Memory Size"]
p_ini = max(0, min(init_orbit*300, total_steps-dP))
p_end = min(p_ini + dP, total_steps)
vC, nvC, idleC = "g", "r", "orange"
action_duration = Config_dict["Action Duration"][1:]
print(f"ini: {p_ini}, end: {p_end}")
print(f"Pos ini: {Telemetry_dict['Pos'][p_ini]/360}, Pos end: {Telemetry_dict['Pos'][p_end]/360}")
print(f"Total Oribts: {tot_orbit}")
print(f"Total Orbit Tele: {t_orbits}")
print(f"Total steps: {total_steps}")
print(f"Action duration: {action_duration}")
print(f"Orbits: {Telemetry_dict['Orbit'][-1]}")

ini: 0, end: 3000
Pos ini: 0.003333333333333333, Pos end: 10.003333333333394
Total Oribts: 33
Total Orbit Tele: 33
Total steps: 9977
Action duration: [20 50 20]
Orbits: 33


## Actions sent

In [11]:
# fig, ax = plotConfig(Config_dict, fig_size=fig_size, y_max = len(Config_dict["action_list"]), epsilon=2,x_min= Telemetry_dict["Pos"][p_ini], x_max=Telemetry_dict["Pos"][p_end])
for i, a in enumerate(Telemetry_dict["action"][p_ini:p_end]):
    i_pos = p_ini + i
    alp = 1.
    
    if (not Telemetry_dict["Busy"][i_pos-1] or i_pos==0):
        if Telemetry_dict["Valid Action"][i]:
            c = vC
        else:
            c = nvC
    else:    
        c = nvC
        alp = .5

    if i_pos==0:
        x_a = 0
    else:
        x_a = Telemetry_dict["Pos"][i_pos-1]/360
    # if a != 0:
        # ax.plot(x_a, a, c, marker="o", markersize=mks, alpha=alp)
# ax.set_yticks([ a for a in range(1, total_actions)])
ia, ip = -1, -1
for i, ac_name in enumerate(Config_dict["action_list"]):
    if i>0:
        if "dump" in ac_name:
            if ia < 0:
                ia = i - 1
            actName = ac_name.replace("dump img", "DP ")
        elif "analyze" in ac_name:
            if ip < 0:
                ip = i - 1
            actName = ac_name.replace("analyze img", "AP ")
        elif "take_picture" in ac_name:
            actName = ac_name.replace("take_picture img", "TP ")
        action_list_str.append(actName)
    else:
        action_list_str = ["idle"]

# ax.set_yticklabels(action_list_str)


# ax2 = ax.twinx()  
# ax2.plot(Telemetry_dict["Pos"]/360, 100*Telemetry_dict["Memory Level"]/memory_size, "b", label="Memory")
# ax2.plot(Telemetry_dict["Pos"]/360, Telemetry_dict["Power"], "g", label="Power")
# ax2.set_yticks([0, 25, 50, 75, 100])
# ax2.set_ylabel("Power / Memory (%)")
# ax2.legend(loc="upper right")
# ax2.set_ylim(0, 100)
# ax2.grid(color='blue', linestyle='--', linewidth=2, alpha=.5)
# ax.grid()
# ax.set_ylim(0, total_actions)
# ax.set_xlabel("Position (Orbits)")
# ax.set_xlim((Telemetry_dict["Pos"][max(0,p_ini-1)]-1)/360, Telemetry_dict["Pos"][p_end]/360)
# fig.savefig(foldername_specific + "DotActions.png", dpi=300)

## Schedule

In [12]:
import matplotlib
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
matplotlib.rcParams.update({'font.size': 22})
fig, ax = plotConfig(Config_dict, fig_size=fig_size, y_max = len(Config_dict["action_list"]), epsilon=2,x_min= Telemetry_dict["Pos"][p_ini], x_max=Telemetry_dict["Pos"][p_end])
ap = 1
x, y, y_v, y_m = [], [], [], []
pos_vec, action_vec = [], []
busytime = 0
for i, a in enumerate(Telemetry_dict["action"][p_ini:p_end]):
    i_pos = p_ini + i
    busytime -= 1
    if busytime <= 0:
        
        mkr = "|-"
        if action_vec != []:
            x.append(pos_vec)
            y.append(action_vec)
            y_v.append(c)
            y_m.append(mkr)
        
        if a!=0:
            if Telemetry_dict["Valid Action"][i_pos]:
                c = vC
            else:
                c = nvC
            busytime = action_duration[(a-1)//total_targets]/2
        else:
            c = idleC
        
        action_vec = []
        pos_vec = []
        
        # print(busytime)
        ap = a

    if i_pos==0:
        x_a = 0
    else:
        x_a = Telemetry_dict["Pos"][i_pos-1]/360
    action_vec.append(ap)
    pos_vec.append(x_a)


for i in range(len(x)):
    if y[i][0] != 0:

        ax.plot(x[i], y[i], y_m[i], color=y_v[i], markersize=20, markevery=[0, -1], linewidth=2)
y_ticks  = [1, ip, ip+1,  ia, ia+1, len(action_list_str)-1]
ax.yaxis.set_minor_locator(AutoMinorLocator(ip-1))

print(y_ticks)
ax.set_yticks(y_ticks)
ylabels = []
for i, a in enumerate(action_list_str):
    if i in y_ticks and i>0:
        if i%2 == 0:
            ylabels.append("\n "+a)
        else:
            ylabels.append(a+"\n")
print(ylabels)
ax.set_yticklabels(ylabels)
fig.suptitle("Test 2: Telemetry and Schedule")

ax.tick_params(which='both', width=2)
ax.tick_params(which='major', length=14)
ax.tick_params(which='minor', length=4,)
ax.grid(which="minor", alpha=.5, linewidth=1)
ax.grid(which="major", alpha=1, linewidth=2)

ax2 = ax.twinx()  
ax2.plot(Telemetry_dict["Pos"]/360, 100*Telemetry_dict["Memory Level"]/memory_size, "b", label="Memory")
ax2.plot(Telemetry_dict["Pos"]/360, Telemetry_dict["Power"], "g", label="Power")
ax2.set_yticks([0, 25, 50, 75, 100])
ax2.set_ylabel("Power / Memory (%)")
ax2.legend(loc="upper left")
ax2.set_ylim(0, 100)
ax2.grid(color='blue', linestyle='--', linewidth=2, alpha=.5)
ax.set_ylim(0, total_actions)
ax.set_xlabel("Position (orbits)")
ax.set_xlim((Telemetry_dict["Pos"][max(0,p_ini-1)]-1)/360, Telemetry_dict["Pos"][p_end]/360)
fig.savefig(foldername_specific + "BracketActions2.png", dpi=300)

1800
[1, 20, 21, 40, 41, 60]
['TP 1\n', '\n TP 20', 'AP 1\n', '\n AP 20', 'DP 1\n', '\n DP 20']


# Goals

In [13]:
p_ini = 0
n_orbits = 300
total_steps = len(Telemetry_dict["Pos"])-1
tot_orbit = total_steps // 300
t_orbits = Telemetry_dict["Orbit"][-1]
dP = int(n_orbits*300)
p_end = min(p_ini + dP, total_steps)

In [15]:
matplotlib.rcParams.update({'font.size': 24})
fig, ax = plt.subplots(1,1, figsize=fig_size)
for k, v in Telemetry_dict.items():
    if "_Goals" in k:
        
        Init_goal = np.sum(v[0])
        k = k.replace("_Goals", "")
        k = k.replace("_", " ")
        goals_left = np.sum(np.stack(v[p_ini:p_end]), axis=1)
         
        for i, g in enumerate(goals_left):
            if g > 0 and (goals_left[i-1] == 0 or i == 0):
                goals_plot = goals_left[i:]
                pos_plot = Telemetry_dict["Pos"][p_ini+i:p_end]/360
                break
        st_en_plot = ax.plot(pos_plot[0], goals_plot[0], label=k, marker="o", markersize=10,linewidth=0)
        color = st_en_plot[0].get_color()
        ax.plot(pos_plot, goals_plot, color=color, linewidth=2)
ax.set_xlabel("Position (orbits)")
ax.set_xlim(Telemetry_dict["Pos"][max(0,p_ini-1)]-1, Telemetry_dict["Pos"][p_end]/360)
done = False

# ax.set_ylim(-.1, 101)
ax.set_ylabel("Goals Left")
fig.suptitle("Test 3: Goal Development")

ax.yaxis.set_minor_locator(AutoMinorLocator(2))
ax.tick_params(which='both', width=2)
ax.tick_params(which='major', length=14)
ax.tick_params(which='minor', length=4,)
ax.grid(which="minor", alpha=.5, linewidth=1)
ax.grid(which="major", alpha=1, linewidth=2)

ax.legend(loc="upper right")
# ax.set_ylim(-.5, 10.5)

fig.savefig(foldername_specific + "Goals.png", dpi=300)